In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

from IPython.display import clear_output

In [2]:
train = pd.read_csv('../data/train.csv', usecols=['ciudad', 'provincia', 'lat', 'lng'])
test = pd.read_csv('../data/test.csv', usecols=['ciudad', 'provincia', 'lat', 'lng'])

In [3]:
geolocator = Nominatim(user_agent="my-application") 

In [4]:
# La siguiente ciudad no es reconocida por el geolocalizador.
# Se completa manualmente.
ciudad = 'Chalchicomula de Sesma'
print(geolocator.geocode(ciudad).latitude)
print(geolocator.geocode(ciudad).longitude)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [36]:
# Se juntan todas las ciudades del train y del test.
aux = train['ciudad']
aux = aux.append(test['ciudad'])

# Se buscan los strings únicos y se ordenan en forma descendente
# por cantidad con value_counts(). Al final se pasa a Dataframe.
aux = aux.value_counts().to_frame()

# Se renombra el índice como 'ciudad' del índice.
aux.rename_axis('ciudad', axis=0, inplace=True)

# Se renombra la columna de catidades como 'lat'.
aux.rename(columns={'ciudad':'lat'}, inplace=True)

# Se agrega una columna 'lng' y 'lat' con igual número de filas
# que el que tiene el DatFrame aux.
aux['lat'] = np.zeros(aux.shape[0], dtype=np.uint8)
aux['lng'] = np.zeros(aux.shape[0], dtype=np.uint8)

##### Obtención de coordenadas en México
La conexión sufre cortes espontáneos que lanzan excepciones. Esta pare es semiautomatizada, se vuelve a iniciar manualmente desde donde se corta.

In [194]:
%%time
ini = 915
fin = 921
i=1
for index in aux.index[ini:fin] :
    clear_output(wait=True)
    dummy = geolocator.geocode(index, country_codes='mx')
    aux.loc[index, 'lat'] = dummy.latitude
    aux.loc[index, 'lng'] = dummy.longitude
    print(i, 'de', fin - ini)
    i = i + 1

6 de 6
CPU times: user 50.8 ms, sys: 13.5 ms, total: 64.3 ms
Wall time: 12.5 s


In [160]:
aux.loc[aux.index[664], :]

lat    18.983333
lng   -97.450000
Name: Chalchicomula de Sesma, dtype: float64

##### Se completa manualmente Chalchicomula de Sesma.

In [159]:
#aux.loc[aux.index[664], 'lat'] = 18.9833333
#aux.loc[aux.index[664], 'lng'] = -97.450000

In [204]:
aux.to_csv('../data/info_coord_ciudades.csv', index=True)

In [205]:
prueba = pd.read_csv('../data/info_coord_ciudades.csv')

In [206]:
prueba

,ciudades,lat,lng
0,Querétaro,20.854258,-99.847560
1,Benito Juárez,20.816967,-98.178268
2,Zapopan,20.703205,-103.426124
3,San Luis Potosí,22.500000,-100.500000
4,Mérida,20.967076,-89.623740
...,...,...,...
916,Ocampo,22.843612,-99.329910
917,Villa de Allende,19.405912,-99.033846
918,El Grullo,19.795548,-104.177816
919,Suchiapa,16.594633,-93.126952


# Coordenadas de provincias

In [23]:
info_provincias = train['provincia'].value_counts().to_frame()
info_provincias.rename(columns={'provincia':'lat'}, inplace=True)
info_provincias.reset_index(inplace=True)
info_provincias.rename(columns={'index':'provincias'}, inplace=True)

diccionario = {
    'Distrito Federal':'Ciudad de México',
    'San luis Potosí':'San Luis Potosí',
    'Edo. de México':'Estado de México',
}
info_provincias.replace(diccionario, inplace=True)
info_provincias['lng'] = np.zeros(info_provincias.shape[0])
info_provincias.set_index('provincias', inplace=True)

info_provincias

,lat,lng
provincias,,
Ciudad de México,58790,0.0
Estado de México,41607,0.0
Jalisco,21238,0.0
Querétaro,16988,0.0
Nuevo León,15324,0.0
Puebla,10421,0.0
San Luis Potosí,8447,0.0
Yucatán,7928,0.0
Morelos,7337,0.0


# Se hace manualmente de a trozos (si no se corta la conexión)

In [25]:
%%time
ini = 0
fin = info_provincias.index.size
i=1
for index in info_provincias.index[ini:fin] :
    clear_output(wait=True)
    dummy = geolocator.geocode(index, country_codes='mx')
    info_provincias.loc[index, 'lat'] = dummy.latitude
    info_provincias.loc[index, 'lng'] = dummy.longitude
    print(i, 'de', fin - ini)
    i = i + 1

32 de 32
CPU times: user 349 ms, sys: 47.9 ms, total: 397 ms
Wall time: 1min 8s


In [35]:
info_provincias.to_csv('../data/info_coord_provincias.csv', index=True)

In [36]:
prueba = pd.read_csv('../data/info_coord_provincias.csv')
prueba

,provincias,lat,lng
0,Ciudad de México,19.432601,-99.133342
1,Estado de México,19.483945,-99.689972
2,Jalisco,20.333333,-103.666667
3,Querétaro,20.854258,-99.847560
4,Nuevo León,26.238436,-99.887300
5,Puebla,18.833333,-98.000000
6,San Luis Potosí,22.500000,-100.500000
7,Yucatán,20.684596,-88.875567
8,Morelos,18.750000,-99.000000
9,Veracruz,19.333333,-96.666667
